In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import networkx as nx
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score


date = "2022-06-01"
order = pd.read_csv("..//..//Database//NYC_trip//order_clean_260.csv", index_col=0)
network_metrics = pd.read_csv("..//..//Database//network_test_0305//network_metrics_%s.csv" % date, index_col=0)
SP = pd.read_csv("..//..//Database//network_test_0305//SP_60_%s.csv" % date, index_col=0)

order["call_time"] = pd.to_datetime(order["call_time"])
order["end_time"] = pd.to_datetime(order["end_time"])
start_time = pd.to_datetime(date + " 06:00:00 AM")
end_time = pd.to_datetime(date + " 12:00:00 PM")
order_one_day = order[(order["call_time"] >= start_time) & (order["call_time"] < end_time)]

SP.columns = ["SP"]
SP = SP.reset_index(drop=True)
order_one_day = order_one_day.sort_values(by="call_time").reset_index(drop=True)
network_metrics = network_metrics.reset_index(drop=True)

# the length of the three dataframes should be the same
assert len(order_one_day) == len(network_metrics) == len(SP)

# the index of the three dataframes should be the same
assert order_one_day.index.equals(network_metrics.index) 
assert SP.index.equals(network_metrics.index)

# for each SP time span x (e.g. x=60, 90, 120), the SP value in (endtime-x, endtime) should be dropped
# for example, if the endtime is 12:00, x = 60, then the SP value in (11:00, 12:00) should be dropped
# we aim to predict the SP value with provided information and the network
df = pd.concat([order_one_day, network_metrics, SP], axis=1).dropna()

In [18]:
df

,sid,call_time,eid,end_time,delta_min,degree,in_degree,out_degree,betweenness_centrality,closeness_centrality,katz_centrality,SP
0,158,2022-06-01 06:00:02,136,2022-06-01 06:06:52,6.833333,185.0,80.0,105.0,0.118363,190355.998035,2.598008e+08,10
1,182,2022-06-01 06:00:09,225,2022-06-01 06:09:51,9.700000,111.0,111.0,0.0,0.196663,0.000000,6.913625e+13,10
2,138,2022-06-01 06:00:09,134,2022-06-01 06:21:29,21.333333,33.0,0.0,33.0,0.000000,0.000000,1.000000e+00,8
3,182,2022-06-01 06:00:17,157,2022-06-01 06:04:21,4.066667,214.0,108.0,106.0,0.040652,54770.781264,6.146981e+03,12
4,136,2022-06-01 06:00:23,227,2022-06-01 06:22:26,22.050000,193.0,112.0,81.0,0.048152,30265.758879,1.649093e+04,8
...,...,...,...,...,...,...,...,...,...,...,...,...
30653,66,2022-06-01 11:59:56,226,2022-06-01 12:11:46,11.833333,117.0,117.0,0.0,0.204018,0.000000,8.249191e+14,1
30654,227,2022-06-01 11:59:56,157,2022-06-01 12:30:06,30.166667,248.0,147.0,101.0,0.161102,230073.550446,2.305636e+11,1
30655,137,2022-06-01 11:59:57,159,2022-06-01 12:16:06,16.150000,102.0,25.0,77.0,0.059616,42300.899313,1.112292e+05,1
30656,85,2022-06-01 11:59:58,157,2022-06-01 12:22:53,22.916667,124.0,49.0,75.0,0.143032,16644.148663,1.701566e+10,1
